In [ ]:
import tensorflow as tf

In [ ]:
num_gpus_avaliable = len(tf.config.experimental.list_physical_devices("GPU"))
print(num_gpus_avaliable)
assert num_gpus_avaliable > 0

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
x= pd.read_csv('final_dataset_for_bais_pwd.csv')
x.head()

In [ ]:
bool_series = pd.isnull(x["label"])

x[bool_series]

In [ ]:
reviews = x['Hate Text'].values.tolist()
labels = x['label'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_lables, validation_labels = train_test_split(reviews, labels, test_size = 0.2)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer([training_sentences[0]], truncation=True, padding=True,max_length = 128 )

In [ ]:
train_encodings = tokenizer(training_sentences, truncation=True, padding=True)
val_encodings = tokenizer(validation_sentences, truncation=True, padding= True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), training_lables))

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings), validation_labels))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.engine import data_adapter

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=4e-4, epsilon=1e-08)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(train_dataset.shuffle(10).batch(2),
          epochs=2,
          batch_size=2,
          validation_data=val_dataset)


NameError: ignored

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=4e-4, epsilon=1e-08)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(train_dataset, epochs=2, validation_data=val_dataset)


Epoch 1/2


ValueError: ignored

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
test_sentence = 'this is a bad day to interact with you'

In [ ]:
predict_input = tokenizer.encode(test_sentence, truncation= True, padding=True, return_tensors = 'tf')

In [ ]:
model.save_pretrained("./sentiment")

In [ ]:
oaded_model = TFDistilBertForSequenceClassification.from_pretrained("sentiment") 

In [ ]:
from keras.models import load_model


In [ ]:
tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])
